In [2]:
import pandas as pd
from utils import print_equals

In [6]:
from pathlib import Path

DATA_DIR = Path("data")
RAW_DIR = DATA_DIR / "raw"
INTERIM_DIR = DATA_DIR / "interim"

# Constants and setup
FILE_PATH = RAW_DIR / 'Brazil-Aligned and Non-Aligned All Presidents.xlsx'
SHEET_NAME = 'Cabinet & Bureaucracy'
AGENCY_COLUMN = 'D'

## Adding categories based on cell color

In [3]:
import openpyxl
from openpyxl import load_workbook

In [17]:
# Loading workbook and sheet
wb = load_workbook(FILE_PATH)
ws = wb[SHEET_NAME]

In [4]:
def get_color_index(cell: openpyxl.cell) -> str:
    """
    Returns the color index of the cell.
    """
    return cell.fill.start_color.index if cell.fill.start_color else None
# Testing
print('get_color_index(ws["D1"]): should be white/null -> (00000000)')
print(get_color_index(ws["D1"]),) # deve ser nulo
print('get_color_index(ws["D16"]): should be yellow -> (FFFFFF00)')
print(get_color_index(ws["D16"])) # deve ser amarelo
print('get_color_index(ws["D257"]): should be red -> (FFFF0000)')
print(get_color_index(ws["D257"])) # deve ser vermelho

colors_dict = {
    "branco": "00000000",
    "amarelo": "FFFFFF00",
    "vermelho": "FFFF0000",
}

get_color_index(ws["D1"]): should be white/null -> (00000000)


NameError: name 'ws' is not defined

In [18]:

def categorize_rows(ws: openpyxl.worksheet, column_letter: str) -> list:
    """
    Iterate through the rows in the column and add a category based
    on the color of the cell
    """
    categories = []
    
    for row in ws.iter_rows():
        cell = row[ord(column_letter.upper()) - ord('A')] # Gets the column number using ord
        color = get_color_index(cell) 
        
        if color == colors_dict["amarelo"]:
            category = "contra"
        elif color == colors_dict["vermelho"]:
            category = "alinhada"
        else:
            category = "neutra"
        
        categories.append(category)
    
    return categories

# Testing
categorized_rows_list = categorize_rows(ws, AGENCY_COLUMN)
# 1: neutra
print(f"line 1: {categorized_rows_list[0]}")
# 16: contra
print(f"line 16: {categorized_rows_list[15]}")
# 257: alinhada
print(f"line 257: {categorized_rows_list[256]}")

line 1: neutra
line 16: contra
line 257: alinhada


## Loading dataset

In [19]:
df = pd.read_excel(FILE_PATH, sheet_name=SHEET_NAME)
df.head(5)
df.to_parquet("brazil-aligned-parquet-raw.parquet")

## Transforming the dataset

In [20]:
data_parquet = RAW_DIR / "brazil-aligned-parquet-raw.parquet"
df = pd.read_parquet(data_parquet)

In [21]:
copy_df = df.copy()

# Removing unnecessary columns
columns = [
    'President',
    'Year',
    'Agency',
    '% Concedico e Parcialmente',
]
copy_df = copy_df[columns]

# Propagating last valid observation in the columns (this will correctly
# fill the President and Year columns)
copy_df.ffill(inplace=True)

# Adding column with category based in the color
categorized_rows_list_copy = categorized_rows_list
categorized_rows_list_copy = categorized_rows_list_copy[1:] # The first row is the header of the df
copy_df["category"] = pd.Series(categorized_rows_list_copy)

print("Testing categories")
print_equals(20)

print(f"Expected: line 0: neutra")
print(copy_df["category"].iloc[0])
print_equals(10)

print(f"Expected: line 154: contra")
print(copy_df["category"].iloc[14])
print_equals(10)

print(f"Expected: line 255: alinhada")
print(copy_df["category"].iloc[255])
print_equals(10)

# Setting year column to int
copy_df["Year"] = copy_df["Year"].astype(int)

# Renaming columns
copy_df.rename(columns={"% Concedico e Parcialmente": "conc_parc"}, inplace=True)
copy_df.columns = copy_df.columns.str.lower()

display(copy_df.head(5))
display(copy_df.tail(5))

Testing categories
Expected: line 0: neutra
neutra
Expected: line 154: contra
contra
Expected: line 255: alinhada
alinhada


,president,year,agency,conc_parc,category
0,Dilma Rousseff,2015,ABGF - Agencia Brasileira Gestora de Fundos Ga...,0.339623,neutra
1,Dilma Rousseff,2015,AEB – Agência Espacial Brasileira,0.962264,neutra
2,Dilma Rousseff,2015,AMAZUL - Amazônia Azul Tecnologias de Defesa S.A.,0.909091,neutra
3,Dilma Rousseff,2015,AN – Arquivo Nacional,0.955128,neutra
4,Dilma Rousseff,2015,ANA – Agência Nacional de Águas,0.996875,neutra


,president,year,agency,conc_parc,category
3204,Luiz Inácio Lula da Silva,2024,MT - Ministério dos Transportes,0.855362,neutra
3205,Luiz Inácio Lula da Silva,2024,MTE - Ministério do Trabalho e Emprego,0.733654,neutra
3206,Luiz Inácio Lula da Silva,2024,MTur - Ministério do Turismo,0.897810,neutra
3207,Luiz Inácio Lula da Silva,2024,SGPR – Secretaria-Geral da Presidência da Repú...,0.828947,neutra
3208,Luiz Inácio Lula da Silva,2024,SRI – Secretaria de Relações Institucionais da...,0.775510,neutra


In [23]:
final_df = copy_df.copy()
final_df.to_parquet(INTERIM_DIR / "data_parquet_final_1.parquet")

### Comparando média de conformidade das agências contra governos populistas e a favor de governos populistas antes, depois e durante o governo bolsonaro

In [7]:
final_df = pd.read_parquet(INTERIM_DIR / "data_parquet_final_1.parquet")

In [8]:
import plotly.express as px
import plotly.graph_objects as go

# Filtering "contra" and "alinhada" agencies
df_plot_1 = final_df.copy()
df_plot_1 = df_plot_1[df_plot_1['category'].isin(['contra', 'alinhada'])]

# Calculating mean of conformity by each year and category
df_plot_1 = df_plot_1.groupby(['year', 'category'])['conc_parc'].mean().reset_index()

# Getting the president of each year
df_presidents = final_df[['year', 'president']].drop_duplicates()
df_plot_1 = df_plot_1.merge(df_presidents, on='year', how='left')

# Criar gráfico
fig = px.line(
    df_plot_1, x='year', y='conc_parc',
    color='category',  # Apenas duas linhas para as categorys
    line_dash='category',
    markers=True, 
    hover_data={'year': True, 'conc_parc': True, 'president': True},
    labels={'conc_parc': 'Média de Conformidade', 'year': 'Ano', 'category': 'category'}
)

# Adicionar pontos específicos para os presidentes
for presidente in df_presidents['president'].unique():
    df_pres = df_plot_1[df_plot_1['president'] == presidente]
    fig.add_trace(go.Scatter(
        x=df_pres['year'], y=df_pres['conc_parc'],
        mode='markers',
        marker=dict(size=10, symbol='circle'),
        name=presidente
    ))


fig.update_layout(
    title='Média de Conformidade das Agências "Contra" e "Alinhadas" ao Longo dos Anos',
    xaxis_title='Ano',
    yaxis_title='Média de Conformidade',
    legend_title='Legenda',
    hovermode='x unified'
)

fig.show()


In [10]:
from plots import plot_grid_all_agencies

fig = plot_grid_all_agencies(final_df, width=1200)
fig.show()

O que mais queria é o seguinte: pega a média das alinhada e das contra só para todos os presidentes que não sejam Bolsonaro. Depois, pega a média para Bolsonaro. Depois compara - test of means (two-tailed T test) - para ver se tem uma diferença estatisticamente significante entre ex. CGU durante presidentes regulares  V. CGU durante Bolsonaro. Faça isso para cada agência, se puder, e depois para todos as agências (alinhada/contra) de forma agregada.